In [1]:
import import_ipynb
import numpy as np
from data_tools import *
from args import parser
import librosa
import os
from tensorflow.keras.models import *
import tensorflow as tf
import matplotlib.pyplot as plt
import soundfile as sf
from IPython.lib.display import Audio

importing Jupyter notebook from data_tools.ipynb
importing Jupyter notebook from args.ipynb


In [2]:
args, unknown = parser.parse_known_args()

In [3]:
#path to find pre-trained weights / save models
voice_weigth_path = args.voice_weigths_folder
noise_weigth_path= args.noise_weigths_folder
#pre trained model
name_model = args.name_model
#directory where read noisy sound to denoise
audio_dir_prediction = args.audio_dir_prediction
#directory to save the denoise sound
dir_save_prediction = args.dir_save_prediction
#Name noisy sound file to denoise
audio_input_prediction = args.audio_input_prediction
#Name of denoised sound file to save
audio_output_voice_prediction = args.audio_output_voice_prediction
audio_output_noise_prediction = args.audio_output_noise_prediction
# Sample rate to read audio
sample_rate = args.sample_rate
# Minimum duration of audio files to consider
min_duration = args.min_duration
#Frame length for training data
frame_length = args.frame_length
# hop length for sound files
hop_length_frame = frame_length
#nb of points for fft(for spectrogram computation)
n_fft = args.n_fft
#hop length for fft
hop_length_fft = args.hop_length_fft
SNR= [0]

In [4]:
for k in range(len(SNR)):
    audio_dir_prediction= args.audio_dir_prediction
    # load json and create model
    json_file = open(voice_weigth_path+'/'+name_model+'= '+str(SNR[k])+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(voice_weigth_path+'/'+name_model+'= '+str(SNR[k])+'.h5')
    
    #Taking test signals according to the SNR
    audio_dir_prediction= new_path(audio_dir_prediction, SNR[k])
    
    #List the test signals
    list_file = os.listdir(audio_dir_prediction)
    list_file= sorted_nicely(list_file)
    length= len(list_file)
    
    for i in range(length):
        name= os.path.splitext(list_file[i])[0]
        audio = audio_files_to_numpy2(audio_dir_prediction, list_file[i], sample_rate,frame_length, hop_length_frame, min_duration)
        dim_square_spec = int(n_fft / 2) + 1

        complex_mat, real_mat, imag_mat= numpy_audio_to_matrix_spectrogram(audio, dim_square_spec,n_fft, hop_length_fft)
        audio_concat= np.concatenate((real_mat, imag_mat))

        #global scaling to have distribution -1/1
        X_in1 = scaled_in(audio_concat)

        #Reshape for prediction
        X_in1 = X_in1.reshape(X_in1.shape[0],X_in1.shape[1],X_in1.shape[2],1)

        #Prediction using loaded network
        X_pred1 = loaded_model.predict(X_in1)

        #Rescale back the noise model
        inv_sca_X_pred1 = inv_scaled_ou(X_pred1)
        X_denoise1 = audio_concat - inv_sca_X_pred1[:,:,:,0]
        #X_denoise1 = inv_sca_X_pred1[:,:,:,0]
        
        l = int(X_denoise1.shape[0]/2)
        h= int(X_denoise1.shape[0])

        real = X_denoise1[0:l,:,:]
        imaginary= X_denoise1[l:h,:,:]

        my_complex_matrix= real+ 1j*imaginary
        mag, phase= librosa.magphase(my_complex_matrix)

        enhanced_speech= matrix_spectrogram_to_numpy_audio(mag, phase, frame_length, hop_length_fft)
        nb_samples= enhanced_speech.shape[0]

        enhanced_speech_new= enhanced_speech.reshape(1, nb_samples*frame_length)
        sf.write('./Evaluation Techniques/composite/Enhanced/'+str(SNR[k])+'/'+name+'Enhanced'+'.wav', enhanced_speech_new[0, :], sample_rate)